In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np

# Veri yollarını tanımla
train_dir = r'E:\Ben\TRYazilim\hair_analysis_test_project\hair_scalp\Hair_Diseases\train'
val_dir = r'E:\Ben\TRYazilim\hair_analysis_test_project\hair_scalp\Hair_Diseases\val'
test_dir = r'E:\Ben\TRYazilim\hair_analysis_test_project\hair_scalp\Hair_Diseases\test'

# Görüntü boyutunu ve toplu boyutunu tanımla
img_height, img_width = 224, 224
batch_size = 32

# Sadece normalleştirme için ImageDataGenerator'lar (veri artırma yok)
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Veri üreteçlerini oluştur
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False # Tahmin yaparken shuffle=False olmalı
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False # Tahmin yaparken shuffle=False olmalı
)

# Sınıf etiketlerini al
class_names = list(train_generator.class_indices.keys())
print("Sınıf Etiketleri:", class_names)


# Önceden eğitilmiş MobileNetV2 modelini yükle
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Temel modeli dondur (eğitilmeyecek)
base_model.trainable = False

# Model oluştur
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation='softmax')  # 10 hastalık sınıfı için
])

# Hiperparametreler
learning_rate = 0.001
epochs = 15
patience = 5  # Erken durdurma için

# Optimizasyon algoritması ve kaybı tanımla
optimizer = optimizers.Adam(learning_rate=learning_rate)
loss_function = 'categorical_crossentropy'
metrics = ['accuracy', 'top_k_categorical_accuracy', "recall", "precision"]

# Modeli derle
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Eğitim ve doğrulama kaybını kaydetmek için callback oluştur
callbacks = [
    keras.callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose = 1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
]

# Eğitim
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=callbacks
)


Found 9600 images belonging to 10 classes.
Found 1200 images belonging to 10 classes.
Found 1200 images belonging to 10 classes.
Sınıf Etiketleri: ['Alopecia Areata', 'Contact Dermatitis', 'Folliculitis', 'Head Lice', 'Lichen Planus', 'Male Pattern Baldness', 'Psoriasis', 'Seborrheic Dermatitis', 'Telogen Effluvium', 'Tinea Capitis']
Epoch 1/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 874ms/step - accuracy: 0.5862 - loss: 1.2818 - precision: 0.8358 - recall: 0.3621 - top_k_categorical_accuracy: 0.8902
Epoch 1: val_loss improved from inf to 0.30864, saving model to best_model.keras
300/300 ━━━━━━━━━━━━━━━━━━━━ 304s 986ms/step - accuracy: 0.5869 - loss: 1.2802 - precision: 0.8362 - recall: 0.3629 - top_k_categorical_accuracy: 0.8905 - val_accuracy: 0.9569 - val_loss: 0.3086 - val_precision: 0.9803 - val_recall: 0.8826 - val_top_k_categorical_accuracy: 0.9992
Epoch 2/50
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 791ms/step - accuracy: 0.9707 - loss: 0.2333 - precision: 0.9888 - recall: 0.9281 - top_k_categor

KeyboardInterrupt: 

In [ ]:

# Eğitim ve doğrulama kaybını çiz
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Çağ')
plt.ylabel('Kayıp')
plt.legend(loc='upper right')
plt.title('Eğitim ve Doğrulama Kaybı')

# Eğitim ve doğrulama doğruluğunu çiz
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Çağ')
plt.ylabel('Doğruluk')
plt.legend(loc='lower right')
plt.title('Eğitim ve Doğrulama Doğruluğu')

plt.show()

# Test seti üzerinde değerlendirme
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Doğruluğu: {test_accuracy:.4f}')

# Modeli kaydet
model.save('mobilenet_keras.keras')